In [1]:
import sys

def config_paths(user, env_name):
    paths = ['',
             '/home/{0}/{1}/.env/bin'.format(user, env_name),
             '/usr/lib/python35.zip',
             '/usr/lib/python3.5',
             '/usr/lib/python3.5/plat-x86_64-linux-gnu',
             '/usr/lib/python3.5/lib-dynload',
             '/home/{0}/{1}/.env/lib/python3.5/site-packages'.format(user, env_name),
             '/home/{0}/{1}/.env/lib/python3.5/site-packages/IPython/extensions'.format(user, env_name),
             '/home/{0}/.ipython']

    for path in paths:
        sys.path.append(path)
        
config_paths('omri', 'my_gpu')

import os
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

TRAIN_DIR = '../'
K = 20
SAMPLE_NUM = 128 * K   
IMG_SIZE = 224

from keras.models import model_from_json




/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
load_model = False

In [3]:
if load_model:
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights("model.h5")

else:
    from keras.applications.resnet50 import ResNet50
    model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                     pooling=None, classes=128)

    last = model.output

    x = Flatten()(last)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(128, activation='softmax')(x)

    model = Model(model.input, preds)

# Train on all data

In [ ]:
def train_on_chunk(model, chunk_number):
    global K
    global TRAIN_DIR
    global SAMPLE_NUM
    global IMG_SIZE
    X_total = np.zeros(shape = (SAMPLE_NUM, IMG_SIZE,IMG_SIZE,3))
    Y_total = np.zeros(shape = (SAMPLE_NUM,))

    for category_ind in range(1,129):
        cagetory_dir = os.path.join(TRAIN_DIR, str(category_ind))

        cur_image_list = os.listdir(cagetory_dir)
        if (chunk_number+1)*K > len(cur_image_list):
            temp_list = cur_image_list[chunk_number*K:]
        else:
            temp_list = cur_image_list[chunk_number*K:(chunk_number+1)*K]
        for im_ind, im_name in enumerate(temp_list):
            im = cv2.imread(os.path.join(cagetory_dir, im_name))
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (IMG_SIZE, IMG_SIZE))
            im = im/255.0

            X_total[(category_ind-1)*K + im_ind] = im
            Y_total[(category_ind-1)*K + im_ind] = category_ind - 1

    random_indices = np.random.permutation(SAMPLE_NUM)
    X_total = X_total[random_indices]
    Y_total = Y_total[random_indices]

    train_num = int(SAMPLE_NUM * 0.7)
    X_train = X_total[:train_num]
    Y_train = Y_total[:train_num]

    X_val = X_total[train_num:]
    Y_val = Y_total[train_num:]

    Y_val = np_utils.to_categorical(Y_val,128)
    Y_train = np_utils.to_categorical(Y_train,128)

    
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer = ModelCheckpoint('model_iter_dropout.h5', verbose=1, save_best_only=True)
    
    epochs = 30
    learning_rate = 0.001
    decay_rate = learning_rate / epochs

    adam = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False, decay=0)
    
    model.compile(loss='categorical_crossentropy',
          optimizer=adam,
          metrics=['accuracy'])
    
    
    
    model.fit(X_train, Y_train, batch_size=32, 
              nb_epoch=epochs,verbose=1,
              validation_data=(X_val, Y_val), callbacks=[earlystopper, checkpointer])
    
    
    
for chunk in range(10):
    train_on_chunk(model, chunk)

In [ ]:
model_json = model.to_json()
with open("model_iter_dropout.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_iter_dropout.h5")